# Tugas Besar 1 CNN

## Eksperimen dengan load model

In [10]:
# Import dependencies
from cnn.cnn import MyCnn
from cnn.layers import Conv2D, Pooling, Dense, Flatten
from typing import List
import numpy as np
import tensorflow as tf
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [11]:
# Constant
IMG_HEIGHT, IMG_WIDTH = 64, 64
TRAIN_IMG_DIR = './dataset/train'
TEST_IMG_DIR = './dataset/test'
SEED = 6459164
SIGMOID_THRESHOLD = 0.5

In [12]:
# Helper Functions
def save_model(model, file_name: str):
    model_file = open(file_name, 'ab')
    pickle.dump(model, model_file)
    model_file.close()


def load_model(file_name: str):
    model_file = open(file_name, 'rb')
    model = pickle.load(model_file)
    model_file.close()

    return model


def load_img_as_train_dataset() -> np.array:
    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        TRAIN_IMG_DIR,
        seed=SEED,
        image_size=(IMG_HEIGHT, IMG_WIDTH)
    )

    imgs = []
    labels = []
    for img_batch, label_batch in train_ds:
        imgs.append(img_batch.numpy())
        labels.append(label_batch.numpy())

    imgs = np.concatenate(imgs)
    labels = np.concatenate(labels)

    return imgs, labels


def load_img_as_test_dataset() -> np.array:
    test_ds = tf.keras.preprocessing.image_dataset_from_directory(
        TEST_IMG_DIR,
        seed=SEED,
        image_size=(IMG_HEIGHT, IMG_WIDTH)
    )

    imgs = []
    labels = []
    for img_batch, label_batch in test_ds:
        imgs.append(img_batch.numpy())
        labels.append(label_batch.numpy())

    imgs = np.concatenate(imgs)
    labels = np.concatenate(labels)

    return imgs, labels


def normalize_img(img: np.array) -> np.array:
    scalev = np.vectorize(lambda x: x / 255.0)
    return scalev(img)


def reorganize_layer(img: np.array) -> List[np.array]:
    input_shape = img.shape
    rows = input_shape[0]
    cols = input_shape[1]
    depth = input_shape[2]

    layers = [np.array([[0] * cols] * rows) for _ in range(depth)]

    for i, row in enumerate(img):
        for j, col in enumerate(row):
            for k, val in enumerate(col):
                layers[k][i][j] = val

    return layers


def interpret_class(res: List[np.array]) -> int:
    return 0 if res[0][0] < SIGMOID_THRESHOLD else 1

In [13]:
# Load Images
imgs, labels = load_img_as_test_dataset()
labels = list(labels)

img_list = []
for img in imgs:
    img_norm = normalize_img(img)
    img_reorganized = reorganize_layer(img)
    img_list.append(img_reorganized)

Found 40 files belonging to 2 classes.


In [14]:
# Load Model
cnn = load_model('./models/ep5-bs100-lr0.1-m0.01-6459164.pkl')

In [15]:
# Predict
res_classes = []
for img, label in zip(img_list, labels):
    res, layers_input = cnn.feed_forward(img)
    res_class = interpret_class(res)
    res_classes.append(res_class)

In [16]:
# Print confusion matrix
confusion_matrix(labels, res_classes)

array([[ 0, 20],
       [ 0, 20]])